<a href="https://colab.research.google.com/github/roberthsu2003/Transformer/blob/main/%E5%AF%A6%E6%88%B0%E9%81%8B%E7%94%A8/QuestionAnswering/%E6%BB%91%E5%8B%95%E7%AD%96%E7%95%A5%E5%AF%A6%E4%BD%9C/qa_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget
#cmrc_eval.py檔會用到這個套件
!pip install nltk
!pip install datasets
!pip install transformers
!pip install evaluate

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=bc6679832f6175e5357bfb511eb8d6421e6e1a39bba1dc73e2e6552dfbb44751
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
import wget
#cmrc_eval.py評估檔內容有修改過
wget.download('https://raw.githubusercontent.com/roberthsu2003/Transformer/refs/heads/main/for_download/cmrc_eval.py')

'cmrc_eval (1).py'

## 基於視窗滑動策略的機器閱讀理解(MRC)
### 載人套件

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForQuestionAnswering,TrainingArguments,Trainer, DefaultDataCollator

### 下載資料集

In [4]:
datasets = load_dataset("roberthsu2003/for_MRC_QA", cache_dir='data')
datasets

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/732 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.78M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26936 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3524 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3493 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 26936
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3524
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3493
    })
})

### 數據處理

In [5]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [7]:
#建立處理token的function
def process_func(examples):
    tokenized_example = tokenizer(
        text = examples["question"],
        text_pair=examples['context'],
        return_offsets_mapping=True,
        return_overflowing_tokens=True,
        stride = 128, #設定重疊的部份
        max_length=512,
        truncation="only_second",
        padding="max_length"
        )
    sample_mapping = tokenized_example.pop("overflow_to_sample_mapping")
    start_positions = []
    end_positions = []
    example_ids = []

    for idx, _ in enumerate(sample_mapping):
        answer = examples['answers'][sample_mapping[idx]] #參考白板比較好理解
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer['text'][0])

        context_start = tokenized_example.sequence_ids(idx).index(1)
        context_end = tokenized_example.sequence_ids(idx).index(None,context_start)-1

        offset = tokenized_example.get("offset_mapping")[idx]

        if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
            #答案不在context內
            start_token_pos = 0
            end_token_pos = 0
        else:
            #由左而右再由右而左找尋答案的index
            token_id = context_start
            while token_id <= context_end and offset[token_id][0] < start_char:
                token_id += 1
            start_token_pos = token_id
            token_id = context_end
            while token_id >= context_start and offset[token_id][1] > end_char:
                token_id -= 1
            end_token_pos = token_id

        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)
        example_ids.append(examples["id"][sample_mapping[idx]])
        #這些程式碼是為了預測使用的
        tokenized_example["offset_mapping"][idx] = [
            (o if tokenized_example.sequence_ids(idx)[k] == 1 else None)
            for k, o in enumerate(tokenized_example["offset_mapping"][idx])
        ]


    tokenized_example["example_ids"] = example_ids
    tokenized_example["start_positions"] = start_positions
    tokenized_example["end_positions"] = end_positions
    return tokenized_example



In [8]:
tokenized_datasets = datasets.map(process_func, batched=True, remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/26936 [00:00<?, ? examples/s]

Map:   0%|          | 0/3524 [00:00<?, ? examples/s]

Map:   0%|          | 0/3493 [00:00<?, ? examples/s]

### 獲取模型輸出

In [12]:
import numpy as np
import collections

def get_result(start_logits, end_logits, exmaples, features):

    predictions = {}
    references = {}

    # example 和 feature的映射
    example_to_feature = collections.defaultdict(list)
    for idx, example_id in enumerate(features["example_ids"]):
        example_to_feature[example_id].append(idx)

    # 最优答案候选
    n_best = 20
    # 最大答案长度
    max_answer_length = 30

    for example in exmaples:
        example_id = example["id"]
        context = example["context"]
        answers = []
        for feature_idx in example_to_feature[example_id]:
            start_logit = start_logits[feature_idx]
            end_logit = end_logits[feature_idx]
            offset = features[feature_idx]["offset_mapping"]
            start_indexes = np.argsort(start_logit)[::-1][:n_best].tolist()
            end_indexes = np.argsort(end_logit)[::-1][:n_best].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offset[start_index] is None or offset[end_index] is None:
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    answers.append({
                        "text": context[offset[start_index][0]: offset[end_index][1]],
                        "score": start_logit[start_index] + end_logit[end_index]
                    })
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["score"])
            predictions[example_id] = best_answer["text"]
        else:
            predictions[example_id] = ""
        references[example_id] = example["answers"]["text"]

    return predictions, references

### 評估函數

In [13]:
from cmrc_eval import evaluate_cmrc

def metirc(pred):
    start_logits, end_logits = pred[0]
    if start_logits.shape[0] == len(tokenized_datasets["validation"]):
        p, r = get_result(start_logits, end_logits, datasets["validation"], tokenized_datasets["validation"])
    else:
        p, r = get_result(start_logits, end_logits, datasets["test"], tokenized_datasets["test"])
    return evaluate_cmrc(p, r)

### 下載模型

In [14]:
model = AutoModelForQuestionAnswering.from_pretrained("google-bert/bert-base-chinese")

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 配置TrainingArguments

In [15]:
args = TrainingArguments(
    output_dir="models_for_qa_slide",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    num_train_epochs=2,
    report_to='none'
)

### Step8 配置Trainer

In [16]:
trainer = Trainer(
    model=model,
    args=args,
    processing_class=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=DefaultDataCollator(),
    compute_metrics=metirc
)

### 模型訓練

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


### 模型预测

In [21]:
from transformers import pipeline

pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)
pipe

Device set to use cuda:0


In [22]:
pipe(question="小明在哪里上班？", context="小明在北京上班")

{'score': 0.9853875041007996, 'start': 3, 'end': 5, 'answer': '北京'}

In [23]:
from huggingface_hub import login
login()

In [24]:
trainer.push_to_hub("roberthsu2003")
#同時要上傳tokenizer
model_name = "roberthsu2003/models_for_qa_slide"
tokenizer.push_to_hub(model_name)

model.safetensors:   0%|          | 0.00/407M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/roberthsu2003/models_for_qa_slide/commit/c2c8559c8be602177901adbabcb8959cd6e2134e', commit_message='Upload tokenizer', commit_description='', oid='c2c8559c8be602177901adbabcb8959cd6e2134e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/roberthsu2003/models_for_qa_slide', endpoint='https://huggingface.co', repo_type='model', repo_id='roberthsu2003/models_for_qa_slide'), pr_revision=None, pr_num=None)